In [2]:
cd ..

/home/is/yuka-ot/dev/dicomplete


/home/is/yuka-ot/dev/dicomplete/.venv/lib/python3.11/site-packages/IPython/core/magics/osm.py:417: UserWarning: using dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


In [3]:
import pandas as pd
import torch
from src.generate import CustomDataset, Trainer, GenerateText
from transformers import T5Tokenizer, T5ForConditionalGeneration

/tmp/ipykernel_1011153/1140959910.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd
/home/is/yuka-ot/dev/dicomplete/.venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


ModuleNotFoundError: No module named 'utils'

In [6]:
df = pd.read_csv('./data/db_data_DISEASE_SIP-3_v202401_2.1.csv')

/tmp/ipykernel_1011153/2261275121.py:1: DtypeWarning: Columns (6,7,8,9,42,43,44,45,47,49,51) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('./data/db_data_DISEASE_SIP-3_v202401_2.1.csv')


In [7]:
df

,id,uuid,appearance,appearance_reliability,appearance_yomi,appearance_yomi_reliability,standard_name,standard_name_reliability,standard_name_yomi,standard_name_yomi_reliability,...,to_meddraj_reliability,type_symptom,type_symptom_reliability,type_diagnosis,type_diagnosis_reliability,type_negation,type_negation_reliability,freq_global,freq_local,source
0,0,fe55650df39660acbe398b99908725d6,副作用,NaN,ふくさよう,D,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,15232.0,NaN,JST;JAMSMDIC
1,1,17eced89ff5b50b688b319d6c59016c3,合併症,NaN,がっぺいしょう,C,-1,C,-1,NaN,...,C,0.0,C,0.0,C,NaN,NaN,14996.0,20.0,MANBYO;JST;JAMSMDIC
2,2,b41877e5d586b1b557850fd91ad9fe5e,糖尿病,NaN,とうにょうびょう,S,糖尿病,S,とうにょうびょう,S,...,S,0.0,C,1.0,C,NaN,NaN,10535.0,20.0,MANBYO;JST;JAMSMDIC;Lilak
3,3,49aec2159aee4caa79922620af8dd5ef,再発,NaN,さいはつ,D,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,9771.0,NaN,JST;JAMSMDIC
4,4,4a5301964900637efac1f17211ac59a2,癌,NaN,がん,S,癌,S,がん,S,...,S,0.0,C,1.0,C,NaN,NaN,6357.0,0.0,MANBYO;JST;JAMSMDIC;Lilak
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
534773,534773,e9545c2e173dbe4ba63ae02a892e8385,ＱＲＳ拡張,NaN,きゅーあーるえすかくちょう,D,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Lilak
534774,534774,e088bf082d1ef9cc8033e12bcf00f976,Ｓｐｌｉｎｔｅｒ出血,NaN,すぷれのーしゅっけつ,D,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Lilak
534775,534775,9cc008373143439e097e1cd80e5453fb,オスラー有痛性結節,NaN,おすらーゆうつうせいけっせつ,D,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Lilak
534776,534776,42f9f68d44410aa2427d0d1f4d1624a2,胸部単純Ｘ線写真の異常,NaN,きょうぶたんじゅんえっくすせんしゃしんのいじょう,D,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Lilak


In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 534778 entries, 0 to 534777
Data columns (total 55 columns):
 #   Column                          Non-Null Count   Dtype  
---  ------                          --------------   -----  
 0   id                              534778 non-null  int64  
 1   uuid                            534778 non-null  object 
 2   appearance                      534778 non-null  object 
 3   appearance_reliability          0 non-null       float64
 4   appearance_yomi                 534778 non-null  object 
 5   appearance_yomi_reliability     481422 non-null  object 
 6   standard_name                   284495 non-null  object 
 7   standard_name_reliability       284492 non-null  object 
 8   standard_name_yomi              284480 non-null  object 
 9   standard_name_yomi_reliability  43138 non-null   object 
 10  appearance_en                   482982 non-null  object 
 11  appearance_en_reliability       482982 non-null  object 
 12  appearance_zh   

In [9]:
df["appearance_yomi_reliability"].value_counts()

appearance_yomi_reliability
D    415953
S     43127
C     17513
A      4312
B       488
E        29
Name: count, dtype: int64

In [10]:
df["to_icd10_reliability"].value_counts()

to_icd10_reliability
D    443725
C     43127
S     43123
A      4315
B       488
Name: count, dtype: int64

In [11]:
df[df["appearance_yomi_reliability"]=="S"]["appearance"].str.len().describe()

count    43127.000000
mean         7.625896
std          3.451737
min          1.000000
25%          5.000000
50%          7.000000
75%          9.000000
max         31.000000
Name: appearance, dtype: float64

In [12]:
df[df["appearance_yomi_reliability"]=="S"]["appearance_yomi"].str.len().describe()

count    43127.000000
mean        13.443991
std          5.769041
min          2.000000
25%         10.000000
50%         13.000000
75%         16.000000
max        218.000000
Name: appearance_yomi, dtype: float64

In [18]:
ranks = "SABCDE"
rank = "A"
input_column="appearance"
target_column="appearance_yomi"
ix_train = ranks.index(rank) + 1  # => 1 + 1 = 2
ranks_train = ranks[:ix_train]    # => "SA"
str_ranks_train = r"|".join(r for r in ranks_train)  # => "S|A"
mask_train = df[f'{target_column}_reliability'].fillna("E").str.contains(str_ranks_train)
df_train = df[mask_train][[input_column, target_column, f'{target_column}_reliability']]
df_test = df[~mask_train][[input_column, target_column, f'{target_column}_reliability']]  # ~mask_train == not mask_train


In [32]:
# 事前学習済みモデル
# MODEL_NAME = "megagonlabs/t5-base-japanese-web"
MODEL_NAME = "retrieva-jp/t5-base-long"
# device = "cuda" if torch.cuda.is_available() else "cpu"

In [33]:
# トークナイザー（SentencePiece）モデルの読み込み
tokenizer = T5Tokenizer.from_pretrained(MODEL_NAME, is_fast=True)

In [34]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [35]:
input_column = "appearance"
target_column = "appearance_yomi"

In [36]:
# データセットの準備
# train_dataset = CustomDataset(df_train.head(10), tokenizer = tokenizer, input_column=input_column, target_column=target_column)

In [37]:
# Trainerの初期化とトレーニング
trainer = Trainer(model_name=MODEL_NAME,tokenizer = tokenizer, train_data = df_train,
                  input_column=input_column, target_column=target_column, batch_size=128,
                  token_max_length_src=16, token_max_length_tgt=16, epochs=50, device = device)
best_model, best_tokenizer, timestamp = trainer.train()

  2%|▏         | 1/50 [02:48<2:17:49, 168.76s/it]

Epoch: 1, Train Loss: 10.04135737293645
Val Accuracy: 0.0009274287039183863, Val Loss: inf
Targets: ['そくふくへきかいほうそう', 'けいぶついかんばんしょうがい', 'にんしんこうけつあつじんしょう', 'せんてんせいえぬえーでぃーえいちめとへもぐろびんかんげんこうそけっそんしょう', 'びせんきていさいぼうがん', 'ちんきゅうせいあっぱくこっせつ', 'こくしょくさもう', 'ふくじんけっかく', 'みゃくらくまくこつしゅ', 'せんてんせいしぼういしゅくしょう', 'はしゅじょうえりてまとーです', 'けいかんたいばん', 'ぜんそうきんぞくぽんぽうはそん', 'ぎじとらこーま', 'しゃそう', 'ほうけいしれつきゅう', 'せんてんせいがいびきけい', 'せめんとしつかんしょく', 'がんめんしんけいまひ', 'かっしょくはくないしょう', 'じょうがくしにくがん', 'そうこうけっそん', 'ちょうせんてんせいゆちゃく', 'けいぶかいりせいどうみゃくりゅう', 'ぶんべんぜんしゅっけつ', 'りょうそくせいはんかいしんけいまひ', 'きゅうせいだいちょうえん', 'やくざいせいていけつあつしょう', 'がいいんぶちつかんじだしょう', 'きんせいぶしんしつちゅうかくけっそん', 'はいぶもうぼしゅ', 'じょうわんこつえんいたんこっせつ', 'こうもんりんきんかんせん', 'とうげんびょうなながた', 'めいそうしんけいしょうがい', 'きょうまくはいえん', 'こうてんせいじんうにょうかんいこうぶきょうさくをともなうすいじんしょう', 'たけいこうはんせいかんせつしょうがい', 'ろうじんせいちついしゅく', 'そうはつせいえーえるぜはいむびょう', 'とくはつせいしゅっけつせいけっしょうばんけっしょうばんけっしょうばんけっしょうばんけっしょうばんけっしょうばんけっしょうばんけっしょうばんけっしょうばんけっしょうばんけっしょうばんけっしょうばんけっしょうばんけっしょうばんけっしょうばんけっしょうばんけっしょうばんけっしょうばんけっしょうばんけっしょうばんけっしょうばんけっしょうばんけっしょうばんけっしょうばんけっしょうばんけっしょうばんけっしょうせ

 12%|█▏        | 6/50 [17:25<2:08:06, 174.69s/it]

Epoch: 6, Train Loss: 1.646290968515371
Val Accuracy: 0.23950846278692325, Val Loss: inf
Targets: ['そくふくへきかいほうそう', 'けいぶついかんばんしょうがい', 'にんしんこうけつあつじんしょう', 'せんてんせいえぬえーでぃーえいちめとへもぐろびんかんげんこうそけっそんしょう', 'びせんきていさいぼうがん', 'ちんきゅうせいあっぱくこっせつ', 'こくしょくさもう', 'ふくじんけっかく', 'みゃくらくまくこつしゅ', 'せんてんせいしぼういしゅくしょう', 'はしゅじょうえりてまとーです', 'けいかんたいばん', 'ぜんそうきんぞくぽんぽうはそん', 'ぎじとらこーま', 'しゃそう', 'ほうけいしれつきゅう', 'せんてんせいがいびきけい', 'せめんとしつかんしょく', 'がんめんしんけいまひ', 'かっしょくはくないしょう', 'じょうがくしにくがん', 'そうこうけっそん', 'ちょうせんてんせいゆちゃく', 'けいぶかいりせいどうみゃくりゅう', 'ぶんべんぜんしゅっけつ', 'りょうそくせいはんかいしんけいまひ', 'きゅうせいだいちょうえん', 'やくざいせいていけつあつしょう', 'がいいんぶちつかんじだしょう', 'きんせいぶしんしつちゅうかくけっそん', 'はいぶもうぼしゅ', 'じょうわんこつえんいたんこっせつ', 'こうもんりんきんかんせん', 'とうげんびょうなながた', 'めいそうしんけいしょうがい', 'きょうまくはいえん', 'こうてんせいじんうにょうかんいこうぶきょうさくをともなうすいじんしょう', 'たけいこうはんせいかんせつしょうがい', 'ろうじんせいちついしゅく', 'そうはつせいえーえるぜはいむびょう', 'とくはつせいしゅっけつせいけっしょうばんけっしょうばんけっしょうばんけっしょうばんけっしょうばんけっしょうばんけっしょうばんけっしょうばんけっしょうばんけっしょうばんけっしょうばんけっしょうばんけっしょうばんけっしょうばんけっしょうばんけっしょうばんけっしょうばんけっしょうばんけっしょうばんけっしょうばんけっしょうばんけっしょうばんけっしょうばんけっしょうばんけっしょうばんけっしょうばんけっしょうせいし

 22%|██▏       | 11/50 [31:30<1:50:03, 169.32s/it]

...

Epoch: 41, Train Loss: 0.08732682145445754
Val Accuracy: 0.38070948295849755, Val Loss: inf
Targets: ['そくふくへきかいほうそう', 'けいぶついかんばんしょうがい', 'にんしんこうけつあつじんしょう', 'せんてんせいえぬえーでぃーえいちめとへもぐろびんかんげんこうそけっそんしょう', 'びせんきていさいぼうがん', 'ちんきゅうせいあっぱくこっせつ', 'こくしょくさもう', 'ふくじんけっかく', 'みゃくらくまくこつしゅ', 'せんてんせいしぼういしゅくしょう', 'はしゅじょうえりてまとーです', 'けいかんたいばん', 'ぜんそうきんぞくぽんぽうはそん', 'ぎじとらこーま', 'しゃそう', 'ほうけいしれつきゅう', 'せんてんせいがいびきけい', 'せめんとしつかんしょく', 'がんめんしんけいまひ', 'かっしょくはくないしょう', 'じょうがくしにくがん', 'そうこうけっそん', 'ちょうせんてんせいゆちゃく', 'けいぶかいりせいどうみゃくりゅう', 'ぶんべんぜんしゅっけつ', 'りょうそくせいはんかいしんけいまひ', 'きゅうせいだいちょうえん', 'やくざいせいていけつあつしょう', 'がいいんぶちつかんじだしょう', 'きんせいぶしんしつちゅうかくけっそん', 'はいぶもうぼしゅ', 'じょうわんこつえんいたんこっせつ', 'こうもんりんきんかんせん', 'とうげんびょうなながた', 'めいそうしんけいしょうがい', 'きょうまくはいえん', 'こうてんせいじんうにょうかんいこうぶきょうさくをともなうすいじんしょう', 'たけいこうはんせいかんせつしょうがい', 'ろうじんせいちついしゅく', 'そうはつせいえーえるぜはいむびょう', 'とくはつせいしゅっけつせいけっしょうばんけっしょうばんけっしょうばんけっしょうばんけっしょうばんけっしょうばんけっしょうばんけっしょうばんけっしょうばんけっしょうばんけっしょうばんけっしょうばんけっしょうばんけっしょうばんけっしょうばんけっしょうばんけっしょうばんけっしょうばんけっしょうばんけっしょうばんけっしょうばんけっしょうばんけっしょうばんけっしょうばんけっしょうばんけっしょうばんけっしょう

 92%|█████████▏| 46/50 [2:07:04<10:59, 164.81s/it]

Epoch: 46, Train Loss: 0.061698362084203644
Val Accuracy: 0.3827961975423139, Val Loss: inf
Targets: ['そくふくへきかいほうそう', 'けいぶついかんばんしょうがい', 'にんしんこうけつあつじんしょう', 'せんてんせいえぬえーでぃーえいちめとへもぐろびんかんげんこうそけっそんしょう', 'びせんきていさいぼうがん', 'ちんきゅうせいあっぱくこっせつ', 'こくしょくさもう', 'ふくじんけっかく', 'みゃくらくまくこつしゅ', 'せんてんせいしぼういしゅくしょう', 'はしゅじょうえりてまとーです', 'けいかんたいばん', 'ぜんそうきんぞくぽんぽうはそん', 'ぎじとらこーま', 'しゃそう', 'ほうけいしれつきゅう', 'せんてんせいがいびきけい', 'せめんとしつかんしょく', 'がんめんしんけいまひ', 'かっしょくはくないしょう', 'じょうがくしにくがん', 'そうこうけっそん', 'ちょうせんてんせいゆちゃく', 'けいぶかいりせいどうみゃくりゅう', 'ぶんべんぜんしゅっけつ', 'りょうそくせいはんかいしんけいまひ', 'きゅうせいだいちょうえん', 'やくざいせいていけつあつしょう', 'がいいんぶちつかんじだしょう', 'きんせいぶしんしつちゅうかくけっそん', 'はいぶもうぼしゅ', 'じょうわんこつえんいたんこっせつ', 'こうもんりんきんかんせん', 'とうげんびょうなながた', 'めいそうしんけいしょうがい', 'きょうまくはいえん', 'こうてんせいじんうにょうかんいこうぶきょうさくをともなうすいじんしょう', 'たけいこうはんせいかんせつしょうがい', 'ろうじんせいちついしゅく', 'そうはつせいえーえるぜはいむびょう', 'とくはつせいしゅっけつせいけっしょうばんけっしょうばんけっしょうばんけっしょうばんけっしょうばんけっしょうばんけっしょうばんけっしょうばんけっしょうばんけっしょうばんけっしょうばんけっしょうばんけっしょうばんけっしょうばんけっしょうばんけっしょうばんけっしょうばんけっしょうばんけっしょうばんけっしょうばんけっしょうばんけっしょうばんけっしょうばんけっしょうばんけっしょうばんけっしょうばんけっしょう

 98%|█████████▊| 49/50 [2:16:38<02:47, 167.32s/it]

Validation loss has not improved for 3 epochs. Training stopped.
Val Accuracy: 0.38349176907025273, Val Loss: inf
Targets: ['そくふくへきかいほうそう', 'けいぶついかんばんしょうがい', 'にんしんこうけつあつじんしょう', 'せんてんせいえぬえーでぃーえいちめとへもぐろびんかんげんこうそけっそんしょう', 'びせんきていさいぼうがん', 'ちんきゅうせいあっぱくこっせつ', 'こくしょくさもう', 'ふくじんけっかく', 'みゃくらくまくこつしゅ', 'せんてんせいしぼういしゅくしょう', 'はしゅじょうえりてまとーです', 'けいかんたいばん', 'ぜんそうきんぞくぽんぽうはそん', 'ぎじとらこーま', 'しゃそう', 'ほうけいしれつきゅう', 'せんてんせいがいびきけい', 'せめんとしつかんしょく', 'がんめんしんけいまひ', 'かっしょくはくないしょう', 'じょうがくしにくがん', 'そうこうけっそん', 'ちょうせんてんせいゆちゃく', 'けいぶかいりせいどうみゃくりゅう', 'ぶんべんぜんしゅっけつ', 'りょうそくせいはんかいしんけいまひ', 'きゅうせいだいちょうえん', 'やくざいせいていけつあつしょう', 'がいいんぶちつかんじだしょう', 'きんせいぶしんしつちゅうかくけっそん', 'はいぶもうぼしゅ', 'じょうわんこつえんいたんこっせつ', 'こうもんりんきんかんせん', 'とうげんびょうなながた', 'めいそうしんけいしょうがい', 'きょうまくはいえん', 'こうてんせいじんうにょうかんいこうぶきょうさくをともなうすいじんしょう', 'たけいこうはんせいかんせつしょうがい', 'ろうじんせいちついしゅく', 'そうはつせいえーえるぜはいむびょう', 'とくはつせいしゅっけつせいけっしょうばんけっしょうばんけっしょうばんけっしょうばんけっしょうばんけっしょうばんけっしょうばんけっしょうばんけっしょうばんけっしょうばんけっしょうばんけっしょうばんけっしょうばんけっしょうばんけっしょうばんけっしょうばんけっしょうばんけっしょうばんけっしょうばんけっしょうばんけっしょうばんけっしょうばんけっしょうばんけっしょ

In [38]:
# best_model = T5ForConditionalGeneration.from_pretrained(
#     "./models/retrieva-jp/t5-base-long/appearance_toappearance_yomi/2024-02-16T19:15:48/model"
# )
# best_tokenizer = T5Tokenizer.from_pretrained(
#     "./models/retrieva-jp/t5-base-long/appearance_toappearance_yomi/2024-02-16T19:15:48/tokenizer"
# )
# timestamp = "2024-02-16T19:15:48"

In [39]:
# テキスト生成の準備
text_generator = GenerateText(best_model, best_tokenizer,df_test, input_column, target_column, timestamp,
                              batch_size=128, token_max_length_src=16, token_max_length_tgt=16, device=device)

# テキスト生成
generated_df = text_generator.generate_text()

  1%|          | 35/3842 [00:55<1:38:32,  1.55s/it]

100%|██████████| 3842/3842 [2:20:22<00:00,  2.19s/it]  


OSError: Cannot save file into a non-existent directory: 'data/outputs'

In [ ]:
generated_df

,appearance,appearance_yomi,appearance_yomi_generated
0,副作用,ふくさよう,ようこう
1,合併症,がっぺいしょう,けっしょう
2,再発,さいはつ,さいはつ
3,高血圧,こうけつあつ,はくしょう
4,痛み,いたみ,しっせつ
...,...,...,...
9995,奇脈,きみゃく,きみゃく
9996,再発大腸癌,さいはつだいちょうがん,さいせいだいちょうがん
9997,二次性アミロイドーシス,にじせいあみろいどーしす,じせいあみろいどー
9998,慢性閉塞性動脈硬化症,まんせいへいそくせいどうみゃくこうかしょう,まんせいへいそくせいどうみゃく
